# Email Sender Application
___

Create an email sender application with a graphical user interface (GUI) that allows users to send emails using their Gmail account. Users can enter their email credentials, recipient(s), subject, and email body, and then send the email through the application.

In [ ]:
# Email Sender: A script that allows the user to send emails with specified recipients, 
# subject, and body, using the SMTP protocol and a user's email credentials.

# Import the necessary libraries:

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

#Define the email sender function:

def send_email(subject, body, to_addr, from_addr, password, filename=None):
    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = from_addr
    msg['To'] = to_addr
    msg['Subject'] = subject
    
    # Add the email body
    msg.attach(MIMEText(body, 'plain'))
    
    # Add the attachment if provided
    if filename:
        attachment = open(filename, "rb")
        p = MIMEBase('application', 'octet-stream')
        p.set_payload((attachment).read())
        encoders.encode_base64(p)
        p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
        msg.attach(p)
    
    # Send the email using SMTP protocol
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(from_addr, password)
    text = msg.as_string()
    server.sendmail(from_addr, to_addr, text)
    server.quit()
    
# Call the email sender function with the required parameters:

send_email(subject="Test Email", body="This is a test email!", to_addr="recipient@example.com", 
           from_addr="sender@gmail.com", password="password", filename="attachment.txt")
 

# Including HTML Content

In [ ]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

sender_email = "my@gmail.com"
receiver_email = "your@gmail.com"
password = input("Type your password and press enter:")

message = MIMEMultipart("alternative")
message["Subject"] = "multipart test"
message["From"] = sender_email
message["To"] = receiver_email

# Create the plain-text and HTML version of your message
text = """\
Hi,
How are you?
Real Python has many great tutorials:
www.realpython.com"""
html = """\
<html>
  <body>
    <p>Hi,<br>
       How are you?<br>
       <a href="http://www.realpython.com">Real Python</a> 
       has many great tutorials.
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )

# Adding Attachments Using the email Package

In [ ]:
import email, smtplib, ssl

from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

subject = "An email with attachment from Python"
body = "This is an email with attachment sent from Python"
sender_email = "my@gmail.com"
receiver_email = "your@gmail.com"
password = input("Type your password and press enter:")

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message["Bcc"] = receiver_email  # Recommended for mass emails

# Add body to email
message.attach(MIMEText(body, "plain"))

filename = "document.pdf"  # In same directory as script

# Open PDF file in binary mode
with open(filename, "rb") as attachment:
    # Add file as application/octet-stream
    # Email client can usually download this automatically as attachment
    part = MIMEBase("application", "octet-stream")
    part.set_payload(attachment.read())

# Encode file in ASCII characters to send by email    
encoders.encode_base64(part)

# Add header as key/value pair to attachment part
part.add_header(
    "Content-Disposition",
    f"attachment; filename= {filename}",
)

# Add attachment to message and convert message to string
message.attach(part)
text = message.as_string()

# Log in to server using secure context and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, text)

### GUI Based

In [1]:
import smtplib
import tkinter as tk
from tkinter import messagebox

# Create a GUI window
window = tk.Tk()
window.title("Email Sender")

# Add labels and entry fields for email credentials, recipient(s), subject, and body
tk.Label(window, text="Email: ").grid(row=0, column=0)
email_entry = tk.Entry(window)
email_entry.grid(row=0, column=1)
tk.Label(window, text="Password: ").grid(row=1, column=0)
password_entry = tk.Entry(window, show="*")
password_entry.grid(row=1, column=1)
tk.Label(window, text="Recipient(s): ").grid(row=2, column=0)
recipient_entry = tk.Entry(window)
recipient_entry.grid(row=2, column=1)
tk.Label(window, text="Subject: ").grid(row=3, column=0)
subject_entry = tk.Entry(window)
subject_entry.grid(row=3, column=1)
tk.Label(window, text="Body: ").grid(row=4, column=0)
body_entry = tk.Text(window, height=10)
body_entry.grid(row=4, column=1)

# Add a function to send the email
def send_email():
    # Get the email credentials, recipient(s), subject, and body from the GUI
    email = email_entry.get()
    password = password_entry.get()
    recipients = recipient_entry.get().split(",")
    subject = subject_entry.get()
    body = body_entry.get("1.0", tk.END)
    
    # Use SMTP to send the email
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as smtp:
            smtp.ehlo()
            smtp.starttls()
            smtp.ehlo()
            smtp.login(email, password)
            message = f"Subject: {subject}\n\n{body}"
            smtp.sendmail(email, recipients, message)
        messagebox.showinfo("Success", "Email sent successfully!")
    except:
        messagebox.showerror("Error", "Failed to send email.")

# Add a button to send the email
send_button = tk.Button(window, text="Send", command=send_email)
send_button.grid(row=5, column=1)

# Start the GUI
window.mainloop()
